In [1]:
#Importing packages and checking GPU
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
from torchsummary import summary
import numpy as np
from PIL import Image,ImageDraw,ImageFont
import os
from sklearn.model_selection import train_test_split

if torch.cuda.is_available():
	print("PyTorch is using the GPU")
	GPUCount = torch.cuda.device_count()
	print(f"Found {GPUCount} GPUs")

	for i in range(GPUCount):
		print(f"GPU {i} found: {torch.cuda.get_device_name(i)}")

	device = torch.device("cuda:0")
else:
	print("PyTorch is using the CPU")
	device = torch.device("cpu")

print(f"Selected Device: {device}")

PyTorch is using the GPU
Found 1 GPUs
GPU 0 found: NVIDIA GeForce RTX 5070 Laptop GPU
Selected Device: cuda:0


In [2]:
# #Load BuiltIn dataset
Transform = transforms.Compose(
	[transforms.ToTensor(),
  transforms.Normalize((0.5,),(0.5,))])
Transform = transforms.Compose(
	[transforms.ToTensor(),
  transforms.Normalize((0.5,),(0.5,))])

# TrainSet = torchvision.datasets.EMNIST(root='./data',train=True,download=True,transform=Transform,split="letters",target_transform=lambda y:y-1)
# TrainImages = TrainSet.data.numpy()
# TrainLabels = TrainSet.targets.numpy() - 1

# TestSet = torchvision.datasets.EMNIST(root='./data',train=False,download=True,transform=Transform,split="letters",target_transform=lambda y:y-1)
# TestImages = TestSet.data.numpy()
# TestLabels = TestSet.targets.numpy() - 1

# print(f"Shape of train data: {TrainImages.shape}")
# print(f"Shape of test data: {TestImages.shape}")

In [ ]:
#Load Custom dataset

FontPath = 'fontsNishant'
Images = []
Labels = []

for Font in os.listdir(FontPath):
	if Font.endswith('ttf'):
		try:
			font = ImageFont.truetype(os.path.join(FontPath,Font),24)
			for i in range(26):
				img = Image.new('L',(28,28),color=0)
				draw = ImageDraw.Draw(img)
				draw.text((8,2),str(chr(ord('A') + i)),font=font,fill=255)
				Images.append(np.array(img))
				Labels.append(i)
		except (OSError,IOError):
			print(f"Could not load font {Font}!")

Images = np.array(Images)
Images = Images.astype("float32") / 255.0
Images = (Images - 0.5) / 0.5
Labels = np.array(Labels)

print(f"Shape of custom data: {Images.shape}")

Could not load font fontsNishant!
Could not load font fontsNishant!
Could not load font fontsNishant!
Shape of custom data: (117390, 28, 28)


In [4]:
#Combine datasets and split into new ones

# Images = np.concatenate((TrainImages,TestImages,Images),axis=0)
# Labels = np.concatenate((TrainLabels,TestLabels,Labels),axis=0)

print(f"Shape of combined data: {Images.shape}")

TrainImages,TestImages,TrainLabels,TestLabels = train_test_split(Images,Labels,test_size=0.2,shuffle=True,stratify=Labels)

print(f"Shape of train data: {TrainImages.shape}")
print(f"Shape of test data: {TestImages.shape}")

class CustomDataset(Dataset):
	def __init__(self,images,labels,transform=None):
		self.images = images
		self.labels = labels
		self.transform = transform

	def __len__(self):
		return len(self.images)
	
	def __getitem__(self, index):
		image = self.images[index]
		label = self.labels[index]

		if self.transform:
			image = self.transform(image)

		return image,label	
	
TrainDataset = CustomDataset(TrainImages,TrainLabels,Transform)
TestDataset = CustomDataset(TestImages,TestLabels,Transform)

TrainLoader = DataLoader(TrainDataset,batch_size=128,shuffle=True)
TestLoader = DataLoader(TestDataset,batch_size=128,shuffle=False)

Shape of combined data: (117390, 28, 28)
Shape of train data: (93912, 28, 28)
Shape of test data: (23478, 28, 28)


In [5]:
#Creating Model

#Define architecture
class EMNISTModel(nn.Module):
	def __init__(self):
		super(EMNISTModel,self).__init__()

		#Layer1
		self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,padding=1,bias=False)#Size does not change
		self.bn1 = nn.BatchNorm2d(num_features=32)
		self.relu1 = nn.ReLU()
		self.pool1 = nn.MaxPool2d(kernel_size=2)#Size halves into 14x14

		#Layer2
		self.conv2 = nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1,bias=False)
		self.bn2 = nn.BatchNorm2d(num_features=32)
		self.relu2 = nn.ReLU()

		#Layer3
		self.conv3 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1,bias=False)
		self.bn3 = nn.BatchNorm2d(num_features=64)
		self.relu3 = nn.ReLU()

		#Layer4
		self.conv4 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1,bias=False)
		self.bn4 = nn.BatchNorm2d(num_features=64)
		self.relu4 = nn.ReLU()

		#Layer5
		self.conv5 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1,bias=False)
		self.bn5 = nn.BatchNorm2d(num_features=64)
		self.relu5 = nn.ReLU()
		self.pool2 = nn.MaxPool2d(kernel_size=2)#Size halves into 7x7

		#FlattenLayer
		self.flatten = nn.Flatten()

		#Layer6
		self.fc1 = nn.Linear(in_features=64*7*7,out_features=64)
		self.relu6 = nn.ReLU()

		#Layer7
		self.fc2 = nn.Linear(in_features=64,out_features=64)
		self.relu7 = nn.ReLU()

		#DropoutLayer
		self.dropout = nn.Dropout(p=0.2)

		#Layer8
		self.fc3 = nn.Linear(in_features=64,out_features=26)

	def forward(self,x):

		#Pass through Layer1
		x = self.pool1(self.relu1(self.bn1(self.conv1(x))))

		#Pass through Layer2
		x = self.relu2(self.bn2(self.conv2(x)))

		#Pass through Layer3
		x = self.relu3(self.bn3(self.conv3(x)))

		#Pass through Layer4
		x = self.relu4(self.bn4(self.conv4(x)))

		#Pass through Layer5
		x = self.pool2(self.relu5(self.bn5(self.conv5(x))))

		#Pass through Layer5
		x = self.flatten(x)

		#Pass through Layer6
		x = self.relu6(self.fc1(x))

		#Pass through Layer7
		x = self.relu7(self.fc2(x))

		#Pass through DropoutLayer
		x = self.dropout(x)

		#Pass through Layer7
		x = self.fc3(x)

		#Return Prediction
		return x

#Create and print summary	
print("Creating EMNIST Model...")
model = EMNISTModel().to(device)
print("Model Created")

print("Model Summary: ")
summary(model,input_size=(1,28,28))

#Compile model
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Creating EMNIST Model...
Model Created
Model Summary: 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             288
       BatchNorm2d-2           [-1, 32, 28, 28]              64
              ReLU-3           [-1, 32, 28, 28]               0
         MaxPool2d-4           [-1, 32, 14, 14]               0
            Conv2d-5           [-1, 32, 14, 14]           9,216
       BatchNorm2d-6           [-1, 32, 14, 14]              64
              ReLU-7           [-1, 32, 14, 14]               0
            Conv2d-8           [-1, 64, 14, 14]          18,432
       BatchNorm2d-9           [-1, 64, 14, 14]             128
             ReLU-10           [-1, 64, 14, 14]               0
           Conv2d-11           [-1, 64, 14, 14]          36,864
      BatchNorm2d-12           [-1, 64, 14, 14]             128
             ReLU-13           [-1, 64, 14, 14] 

In [6]:
#Training and saving the model

print("Training Model...")
epochs = 100
patience = 5
counter = 0
MaxAccuracy = float('-inf')

for epoch in range(epochs):
	#Train Loop

	#Set model to training mode
	model.train()
	TrainLoss = 0.0

	for i,data in enumerate(TrainLoader,0):
		inputs,labels = data[0].to(device),data[1].to(device)

		#Zero the parameter gradients
		optimizer.zero_grad()

		#Forward pass
		outputs = model(inputs)

		#Calculate loss
		loss = loss_function(outputs,labels)

		#Backward pass
		loss.backward()

		#Update weights
		optimizer.step()

		#Update loss
		TrainLoss += loss.item()
	
	#Validation Loop
	model.eval()
	ValidationLoss = 0.0
	correct = 0
	total = 0

	with torch.no_grad():
		for data in TestLoader:
			images,labels = data[0].to(device),data[1].to(device)
			outputs = model(images)
			loss = loss_function(outputs,labels)
			ValidationLoss += loss.item()

			_,predicted = torch.max(outputs.data,1)
			total += labels.size(0)
			correct += (predicted == labels).sum().item()

		acc = 100*correct/total
		print(f"Epoch: {epoch+1}/{epochs} | "
			f"Training Loss: {TrainLoss/len(TrainLoader):.3f} | "
			f"Validation Loss: {ValidationLoss / len(TestLoader):.3f} | "
			f"Accuracy: {acc:.3f}%")
		
		if acc > MaxAccuracy:
			MaxAccuracy = acc
			counter = 0
			torch.save(model.state_dict(),"EMNISTModel.pth")
			print(f"Accuracy has improved at epoch {epoch+1}. Model Saved!")
		else:
			counter += 1
			print(f"Accuracy has not improved. Patience:{counter}/{patience}")
		
		if counter >= patience:
			print("Early stopping triggered!")
			break
		
print("Training Complete")


Training Model...


KeyboardInterrupt: 

In [ ]:
#Testing Model

print("Testing model with inbuilt dataset...")


		#Update loss
model.load_state_dict(torch.load("EMNISTModel.pth"))

TestLoss = 0.0
correct = 0
total = 0

with torch.no_grad():
	for images, labels in TestLoader:

		#Move data to device
		images,labels = images.to(device),labels.to(device)

		#Forward pass
		outputs = model(images)

		#Calculate loss
		loss = loss_function(outputs,labels)
		TestLoss += loss.item()

		#Get predicted class
		_,predicted = torch.max(outputs.data,1)

		#Update total and correct counts
		total += labels.size(0)
		correct += (predicted == labels).sum().item()
	
#Test Results

FinalLoss = TestLoss/len(TestLoader)
FinalAcc = 100*correct/total

print(f"Test Accuracy: {FinalAcc:.3f}%")
print(f"Test Loss: {FinalLoss:.3f}")

Testing model with inbuilt dataset...
Test Accuracy: 71.754%
Test Loss: 0.955
